In [21]:
import pandas as pd
import time
import os, gc
import numpy as np
import re

In [22]:
%%time
#train_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_train.csv")
#test_df=pd.read_csv("../KB_NLP/morphs/komo_morphs_test.csv")
train_df=pd.read_csv("../KB_NLP/raw_data/train.csv")
test_df=pd.read_csv("../KB_NLP/raw_data/public_test.csv")

CPU times: user 915 ms, sys: 68.9 ms, total: 984 ms
Wall time: 983 ms


In [23]:
from konlpy.tag import Okt
okt = Okt()

In [24]:
def okt_morphs(sent):
    sent = okt.morphs(sent)
    return sent

def replacement(df):
    df['okt'] = df['okt'].replace("'","").replace("[","").replace("]","").replace(",","")

In [25]:
aa = train_df.iloc[:10,:]

In [27]:
%%time
aa['okt'] = aa['text'].apply(lambda x: okt_morphs(x))


CPU times: user 126 ms, sys: 290 µs, total: 126 ms
Wall time: 63.1 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
aa

,id,year_month,text,smishing,okt
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[XXX, 은행, 성산, XXX, 팀, 장, 입니다, ., 행복한주말, 되세요]"
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 도, 많이, 웃으시는, 하루, 시작, 하세요, XXX, 은행, 진, 월동,..."
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕하십니까, 고객, 님, ., XXX, 은행, 입니다, ., 금일, 납부, 하셔..."
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[XXX, 고객, 님, 안녕하세요, XXX, 은행, XXX, 지점, 입니다지난, 한..."
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[1월, 은, 새, 로움, 이, 가득, XXX, 입니다, ., 올, 한해, 더, 많..."
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복한주말, 보내세요, XXX, 용, 현남, 전담, 직원, 대리, XXX, 올림]"
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[XXX, 고객, 님, 안녕하세요, XXX, 은행, 무교, 지점, XXX, 과장, ..."
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[XXX, 고객, 님, 지난, 한해, 베풀어, 주신, 은혜, 진심, 으로, 감사, ..."
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설연휴, 가족, 들, 과, 훈훈한, 정, 나누시고, 정겨운, 추억, 많이, 만드세..."
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1,"[(, 광고, ), XXXBaXXX, 고객, 님, 들, 뒤, 엔, XXX, 언제나,..."


In [ ]:
aa

In [2]:
train_morphs = []
for i in tqdm(range(len(train_df))):
    morph = okt.morphs(train_df['text'][i])
    train_morphs.append(morph)    

100%|█████████████████████████████████████████████████████████████████████████| 295945/295945 [31:02<00:00, 158.90it/s]


In [ ]:
for i in tqdm(range(len(train_df))):
    train_df['morphs'][i] = train_df['morphs'][i].replace("'","").replace("[","").replace("]","").replace(",","").replace(".","[SEP]")

In [8]:
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        try:
            len(cjj)
        except:
            return ' '
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '' for c in cjj)
        return cjj_

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'
#def jamo_sentence(sent):
#    sent = sent.replace('XXX', 'X')
#    def transform(char):
#        if char == ' ':
#            return char
#        elif char == 'X':
#            return ' X'
#        elif char == '.':
#            return '. '
#        else:
#            return char##

#    sent_ = ''.join(transform(char) for char in sent)
#    sent_ = doublespace_pattern.sub(' ', sent_)
#    return sent_

jamo_sentence(train_df.loc[0, 'text'])

' ㅇㅡㄴㅎㅐㅇㅅㅓㅇㅅㅏㄴ ㅌㅣㅁㅈㅏㅇㅇㅣㅂㄴㅣㄷㅏ ㅎㅐㅇㅂㅗㄱㅎㅏㄴㅈㅜㅁㅏㄹㄷㅚㅅㅔㅇㅛ'

In [9]:
test_df['jamo'] = test_df['text'].apply(lambda x: jamo_sentence(x))
train_df['jamo'] = train_df['text'].apply(lambda x: jamo_sentence(x))

In [10]:
raw_corpus_fname = '\n'.join(pd.concat([train_df['jamo'], test_df['jamo']]))
file=open('../KB_NLP/jamo_corpus.txt','w')
file.write(raw_corpus_fname)
file.close()

In [2]:
import fasttext

In [11]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 2
                                    
                                )

CPU times: user 17min 56s, sys: 1.12 s, total: 17min 57s
Wall time: 9min 1s


In [12]:
model.save_model("../KB_NLP/bigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [13]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 1
                                    
                                )

CPU times: user 17min 49s, sys: 1.19 s, total: 17min 50s
Wall time: 8min 57s


In [14]:
model.save_model("../KB_NLP/unigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [15]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 3
                                    
                                )

CPU times: user 17min 47s, sys: 1.3 s, total: 17min 48s
Wall time: 8min 57s


In [16]:
model.save_model("../KB_NLP/trigram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [17]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 4
                                    
                                )

CPU times: user 17min 40s, sys: 2.56 s, total: 17min 42s
Wall time: 8min 53s


In [18]:
model.save_model("../KB_NLP/4gram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")

In [19]:
%%time
model = fasttext.train_unsupervised('../KB_NLP/jamo_corpus.txt', 
                                    model='skipgram' ,
                                    loss = 'hs',        # hinge loss
                                    ws=1,               # window size
                                    lr = 0.01,          # learning rate
                                    dim = 200,          # embedding dimension
                                    epoch = 5,          # num of epochs
                                    min_count = 10,     # minimum count of subwords
                                    # encoding = 'utf-8', # input file encoding
                                    thread = 2,          # num of threads
                                    wordNgrams = 5
                                    
                                )

CPU times: user 17min 51s, sys: 3.12 s, total: 17min 54s
Wall time: 8min 59s


In [20]:
model.save_model("../KB_NLP/5gram_model_file.bin")
#model.save_model("/content/drive/My Drive/model_file.bin")